# Network/Flow Accumulation Covariates
Covariates to be calculated along across the flow accumulation grids:
1. fac_glacier_per - % Glacier along flow accumulation = glacier weighted fac/fac
2. fac_lake_per	- % Lake along flow accumulation = lake weighted fac/fac
3. fac_mn_slope - Mean slope contributing area = slope weighted fac/fac
4. fac_north_per - % North along flow accumulation	north weighted fac/fac
5. fac_wetland_per - % Wetland along flow accumulation = wetland weighted fac/fac
6. fac_mn_lcld_xxxx - Mean day of year for LCLD (by year) along flow accumulation = mdy(year) weighted fac/fac

In [1]:
###
# Import modules
import arcpy
import sys
import os

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'sys paths {sys.path}')

imports complete
sys paths ['C:\\Users\\dwmerrigan\\Documents\\Github\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\Github\\AKSSF', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Program Files\\ArcGIS\\Pro\\bin', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\future-0.18.2-py3.7.egg', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\pytz-2020.1-py3.7.egg', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\

## Collect datasets
Store the following in a data dictionary using region as the key
1. Flow accumulation grids
2. Slope - Slope in degrees - float
3. Wetlands - Binary raster data
4. Lakes/Ponds - Feature class convert to raster and
5. Glaciers - Binary raster data
6. North - Binary raster data


In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [3]:
arcpy.env.overwriteOutput = True

# Set data dir equal to directory containing the AKSSF regional sub-folders.
data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

# Regions to be processed
rList = ['Kodiak','Prince_William_Sound','Copper_River','Bristol_Bay','Cook_Inlet']
regions = arcpy.ListWorkspaces(workspace_type="Folder")
regions = [os.path.normpath(r) for r in regions if os.path.basename(r) in rList]

#working gdb to save intermediate outputs
ca_scratchgdb = r"D:\\ca_Outputs\\ca_scratch.gdb"
ca_scratchfol = r"D:\\ca_Outputs"
print (regions)

#Create rvbd scratch folders if they do not already exist
import os, arcpy

ca_scratchfol = r"D:\\ca_Outputs"
if not arcpy.Exists(ca_scratchfol):
    os.mkdir(ca_scratchfol)
    print(f'Creating scratch folder {ca_scratchfol}')
else:
    print(f'Scratch folder {ca_scratchfol} already exists')
print('----------')

if not arcpy.Exists(ca_scratchgdb):
    arcpy.CreateFileGDB_management(ca_scratchfol,'rvbd_scratch.gdb')
    print(f'Creating scratch gdb {ca_scratchgdb}')
else:
    print(f'Scratch folder {ca_scratchgdb} already exists')
print('----------')

#Create Regional output folders
for region in regions:
    rname = os.path.basename(region)
    cascratchpath = os.path.join(ca_scratchfol, rname + '_RVBD')
    if not arcpy.Exists(cascratchpath):
        os.mkdir(cascratchpath)
        print (f'{rname} does not have a scratch folder')
        print (f'Creating contributing area scratch folder at  {cascratchpath}')
    else:
        print(f'Scratch folder for {rname} already created at {cascratchpath}')

    print('----------')
print(f'All scratch workspaces set')

['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']
Scratch folder D:\\ca_Outputs already exists
----------
Creating scratch gdb D:\\ca_Outputs\\ca_scratch.gdb
----------
Scratch folder for Bristol_Bay already created at D:\\ca_Outputs\Bristol_Bay_RVBD
----------
Scratch folder for Cook_Inlet already created at D:\\ca_Outputs\Cook_Inlet_RVBD
----------
Scratch folder for Copper_River already created at D:\\ca_Outputs\Copper_River_RVBD
----------
Scratch folder for Kodiak already created at D:\\ca_Outputs\Kodiak_RVBD
----------
Scratch folder for Prince_William_Sound already created at D:\\ca_Outputs\Prince_William_Sound_RVBD
----------
All scratch workspaces set


In [4]:
import arcpy, os
from arcpy.sa import *
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# set pp factor
arcpy.env.parallelProcessingFactor = "80%"

# Create glacier and nhd lake pond rasters to extract from
glacrast = os.path.join(ca_scratchfol,'glaciers.tif')
lakerast = os.path.join(ca_scratchfol,'lakes.tif')
lakereclass = os.path.join(ca_scratchfol,'lakes_reclass.tif')
glacreclass = os.path.join(ca_scratchfol,'glaciers_reclass.tif')

#list to store glacier fcs
glaciers = []

walk = arcpy.da.Walk(data_dir, topdown = True, datatype='FeatureClass')
for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        # Set glacier polygon fc
        if 'glaciers' == filename:
            glacfc_path = os.path.join(dirpath, filename)
            glaciers.append(glacfc_path)

# Merge glacier FCs in memory
glac_merge = arcpy.Merge_management(glaciers,r'memory\glac_merge')

# Set NHD lakes ponds
lakes = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_NHD_LakesPonds"

if not arcpy.Exists(glacreclass):
    print(f'Creating glacier raster for entire region at {glacrast}')
    arcpy.PolygonToRaster_conversion(glac_merge,'O1Region',glacrast)
    arcpy.ddd.Reclassify(glacrast, "Value", "1 1;NODATA 0", glacreclass, "DATA")
else:
    print(f'Glacier raster exists at {glacreclass}')

if not arcpy.Exists(lakereclass):
    print(f'Creating Lakes raster for entire region at {lakerast}')
    arcpy.PolygonToRaster_conversion(lakes,'FType',lakerast)
    arcpy.ddd.Reclassify(lakerast, "Value", "390 1;NODATA 0", lakereclass, "DATA")
else:
    print(f'Lakes raster exists at {lakereclass}')

Glacier raster exists at D:\\ca_Outputs\glaciers_reclass.tif
Lakes raster exists at D:\\ca_Outputs\lakes_reclass.tif


### Set data dictionary and create/modify inputs as necessary

In [16]:
regions

['D:\\GIS\\AKSSF\\Bristol_Bay',
 'D:\\GIS\\AKSSF\\Cook_Inlet',
 'D:\\GIS\\AKSSF\\Copper_River',
 'D:\\GIS\\AKSSF\\Kodiak',
 'D:\\GIS\\AKSSF\\Prince_William_Sound']

In [5]:
from arcpy.sa import *

lakereclass = os.path.join(ca_scratchfol,'lakes_reclass.tif')
glacreclass = os.path.join(ca_scratchfol,'glaciers_reclass.tif')

# List to store fac grids for data that need to be iterated on grid by grid basis (mdy LCLD)
fac_list = []

caDict = {}

# Walk through gdbs and set input data sets. Create necessary raster datasets if they do not already exist.
akssf_gdb = r'D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb'

for region in regions:
    rname = os.path.basename(region)
    print(rname)
    arcpy.env.workspace = region
    arcpy.env.snapRaster = os.path.join(region, 'fac.tif')
    arcpy.env.cellSize = os.path.join(region, 'fac.tif')
    arcpy.env.mask = os.path.join(region, 'fac.tif')
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    outgdb = gdb[0]
    walk = arcpy.da.Walk(region, topdown = True, datatype=['RasterDataset','FeatureClass'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            # Set glacier polygon fc
            if 'glaciers' == filename:
                glacfc_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,glacfc_path)

            # # Set stream polygon fc
            # elif 'streams_merge' == filename or 'NHDFlowline_merge' == filename:
            #     strsource = os.path.join(dirpath, filename)
            #     append_value(caDict,rname,strsource)


            # # Set merged watersheds dataset
            # elif 'wtds_merge' == filename:
            #     wtdsource = os.path.join(dirpath, filename)
            #     # Dissolve Watersheds - keep as single part
            #     wtd_dis = os.path.join(outgdb,rname + '_wtd_dis')
            #     if not arcpy.Exists(os.path.join(outgdb,rname + '_wtd_dis')):
            #         arcpy.Dissolve_management(wtdsource,wtd_dis,None, None, "SINGLE_PART", "DISSOLVE_LINES")
            #         # Append dissolved watersheds to Dictionary to use as input for rvbd tool
            #     append_value(caDict,rname,wtd_dis)

            # # Set elev raster
            # elif 'elev.tif' == filename:
            #     elev_path = os.path.join(dirpath, filename)
            #     append_value(caDict,rname,elev_path)

            # Set ca raster
            elif 'fac.tif' == filename:
                fac_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,fac_path)
                fac_list.append(fac_path)

            # Set slope raster
            elif 'slope.tif' == filename:
                slope_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,slope_path)

            # Set North raster
            elif 'north.tif' == filename:
                nor_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,nor_path)

            # Set wetland raster
            elif 'wetlands.tif' == filename:
                wras_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,wras_path)

    # Check for lakes raster
    lake_path = os.path.join(region, 'lakes.tif')
    if not arcpy.Exists(lake_path):
        print('Lake raster not yet created')
        lake_mask = arcpy.sa.ExtractByMask(lakereclass, fac_path)
        lake_mask.save(lake_path)
    else:
        print(f'Lake raster {lake_path} already created...')

    # Check for glacier raster
    glac_path = os.path.join(region, 'glaciers.tif')
    if not arcpy.Exists(os.path.join(region, 'glaciers.tif')):
        print('Glacier raster not yet created')
        glacier_mask = arcpy.sa.ExtractByMask(glacreclass, fac_path)
        glacier_mask.save(glac_path)
    else:
        print(f'Glacier raster {glac_path} already created...')

    print(f'Flow accumulation raster input = {fac_path} and exists = {arcpy.Exists(fac_path)}')
    print(f'Slope input = {slope_path} and exists = {arcpy.Exists(slope_path)}')
    print(f'North raster input = {nor_path} and exists = {arcpy.Exists(nor_path)}')
    print(f'Wetlands raster input = {wras_path} and exists = {arcpy.Exists(wras_path)}')
    print(f'Lakes raster input = {lake_path} and exists = {arcpy.Exists(lake_path)}')
    print(f'Glaciers raster input = {glac_path} and exists = {arcpy.Exists(glac_path)}')
    print('----------')
print('----------')
print('Data Dictionary created')

Bristol_Bay
Lake raster not yet created
Glacier raster not yet created
Flow accumulation raster input = D:\GIS\AKSSF\Bristol_Bay\fac.tif and exists = True
Slope input = D:\GIS\AKSSF\Bristol_Bay\slope.tif and exists = True
North raster input = D:\GIS\AKSSF\Bristol_Bay\north.tif and exists = True
Wetlands raster input = D:\GIS\AKSSF\Bristol_Bay\wetlands.tif and exists = True
Lakes raster input = D:\GIS\AKSSF\Bristol_Bay\lakes.tif and exists = True
Glaciers raster input = D:\GIS\AKSSF\Bristol_Bay\glaciers.tif and exists = True
----------
Cook_Inlet
Lake raster not yet created
Glacier raster not yet created
Flow accumulation raster input = D:\GIS\AKSSF\Cook_Inlet\fac.tif and exists = True
Slope input = D:\GIS\AKSSF\Cook_Inlet\slope.tif and exists = True
North raster input = D:\GIS\AKSSF\Cook_Inlet\north.tif and exists = True
Wetlands raster input = D:\GIS\AKSSF\Cook_Inlet\wetlands.tif and exists = True
Lakes raster input = D:\GIS\AKSSF\Cook_Inlet\lakes.tif and exists = True
Glaciers raster

### Begin Creating weighted fac grids
Can these be run in parallel?
